In [1]:
import pandas as pd
df = pd.read_excel('D:\Fin_websites\care_only.xlsx')
company_names = list(df['Company Name'])
company_names = list(set(company_names))
cnt = 0

In [2]:
fin_keys = []
for i in company_names:
    fin_key = ''
    txt_split = i.split(' ')
    for j in range(len(txt_split)):
        if (txt_split[j] == 'PRIVATE' or txt_split[j] == 'LIMITED'):
            txt_split = txt_split[:j]
            break
    if (len(txt_split) >= 4):
        fin_key = '%20'.join(txt_split[:3]).lower()
    else:
        fin_key = '%20'.join(txt_split).lower()
    fin_keys.append(fin_key)

In [3]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import requests
from fake_useragent import UserAgent
import json

def get_info(url):
    ua = UserAgent()
    headers = {'User-Agent': ua.random}
    response = requests.get(url, headers=headers)
    #print(response.text)

    data_string = response.text
    data = json.loads(data_string)
    company_name = data["data"][0]["CompanyID"]
    #print(company_name)

    fin_url = 'https://www.careratings.com/search?Id=' + company_name

    driver = webdriver.Chrome("D:\Fin_websites\chromedriver.exe")
    driver.get(fin_url)
    driver.maximize_window()
    time.sleep(5)
    card = driver.find_element(By.CLASS_NAME, "card-body")
    date = card.find_element(By.TAG_NAME , 'span')
    link = driver.find_element(By.XPATH , "//div[@class='item latest slick-slide slick-current slick-active']")
    first_link = link.find_element(By.TAG_NAME , 'a').get_attribute('href')
    fin_list = [first_link,date.text]
    
    driver.quit()
    return fin_list

In [6]:
not_happen = []

In [24]:
import io
import requests
from PyPDF2 import PdfReader, PdfWriter
import os

def download_and_save_pdf(company_name, date, url):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}

    try:
        response = requests.get(url=url, headers=headers, timeout=120)
        response.raise_for_status()  # Raise an exception for non-2xx status codes
        on_fly_mem_obj = io.BytesIO(response.content)
        pdf_file = PdfReader(on_fly_mem_obj)

        # Create the parent directory if it doesn't exist
        parent_dir = "Care_Ratings"
        os.makedirs(parent_dir, exist_ok=True)

        # Create the subdirectory within the parent directory
        company_dir = os.path.join(parent_dir, company_name)
        os.makedirs(company_dir, exist_ok=True)

        # Save the PDF in the subdirectory
        output_path = os.path.join(company_dir, company_name + ' - ' + date + '.pdf')
        with open(output_path, "wb") as pdf_output:
            pdf_writer = PdfWriter()
            pdf_writer.write(on_fly_mem_obj)
            pdf_output.write(response.content)

        print(f"PDF saved successfully: {output_path}")

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while downloading the PDF: {e}")
    except Exception as e:
        print(f"An error occurred while processing the PDF: {e}")

In [28]:
for i in range(len(fin_keys)):
    try:
        url = 'https://www.careratings.com/header/searchlist?cinput=' + fin_keys[i]
        list1 = get_info(url)
        link = list1[0]
        date = list1[1]
        download_and_save_pdf(company_names[i],date,link)
        print(list1)
    except:
        not_happen.append(company_names[i])

C:\Users\abhiram.intern\AppData\Local\Temp\ipykernel_17808\2745599438.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("D:\Fin_websites\chromedriver.exe")


PDF saved successfully: Care_Ratings\PROODLE HOSPITALITY SERVICES PRIVATE LIMITED\PROODLE HOSPITALITY SERVICES PRIVATE LIMITED - 9TH JANUARY 2024.pdf
['https://www.careratings.com/upload/CompanyFiles/PR/202401130137_Proodle_Hospitality_Services_Private_Limited.pdf', '9TH JANUARY 2024']
PDF saved successfully: Care_Ratings\SHEMAROO ENTERTAINMENT LIMITED\SHEMAROO ENTERTAINMENT LIMITED - 16TH FEBRUARY 2024.pdf
['https://www.careratings.com/upload/CompanyFiles/PR/202402140236_Shemaroo_Entertainment_Limited.pdf', '16TH FEBRUARY 2024']
PDF saved successfully: Care_Ratings\SMILAX LABORATORIES LIMITED\SMILAX LABORATORIES LIMITED - 15TH SEPTEMBER 2023.pdf
['https://www.careratings.com/upload/CompanyFiles/PR/202309120908_Smilax_Laboratories_Limited.pdf', '15TH SEPTEMBER 2023']
PDF saved successfully: Care_Ratings\SUNCITY BUILDCON PRIVATE LIMITED\SUNCITY BUILDCON PRIVATE LIMITED - 1ST APRIL 2024.pdf
['https://www.careratings.com/upload/CompanyFiles/PR/202404140423_Suncity_Buildcon_Private_Limited

In [30]:
def write_list_to_file(list_items, file_name):
    with open(file_name, 'w') as file:
        for item in list_items:
            file.write(str(item) + '\n')
    print(f"List written to '{file_name}' successfully.")

write_list_to_file(not_happen, 'not_happen_care_list.txt')

List written to 'not_happen_care_list.txt' successfully.
